- DB 서버의 정보
    - 주소
    - port
    - user명
    - 비밀번호
    - 데이터베이스명

- 서버의 정보들은 외부에 노출이 되면 보안적으로 문제 발생할 수 있다.
    - 서버의 정보와 같이 외부에 노출이 되는 데이터는 dotenv를 이용하여 데이터를 숨겨서 사용(환경변수 사용)

In [60]:
# dotenv 라이브러리 설치
# !pip install python-dotenv

In [61]:
import pandas as pd
import pymysql
import os
from dotenv import load_dotenv

In [62]:
load_dotenv()

True

In [63]:
# 등록된 환경변수에 접근
# 획득하다 get + 환경변수 env
# getenv {변수명} -> return data의 type이 str
os.getenv('db_name')

'multicam'

In [64]:
_db = pymysql.connect(
    host = os.getenv('host'),
    port = int(os.getenv('port')),
    user = os.getenv('user'),
    password = os.getenv('pw'),
    db = os.getenv('db_name')
)

In [65]:
cursor = _db.cursor(pymysql.cursors.DictCursor)

In [66]:
create_user_query = """
    CREATE TABLE if not exists `user_info`
    (
        `id` varchar(35) primary key,
        `password` varchar(45) not null,
        `name` varchar(45),
        `birth` varchar(10)
    )
"""
cursor.execute(create_user_query)

0

In [69]:
# query문 안에 유저가 입력한 데이터를 대입하여 quert 실행
# 회원가입을 설정
# 입력데이터가 4개 id, password, name, birth
input_id = input('id를 입력하시오')
input_pass = input('password를 입력하시오')
input_name = input('name을 입력하시오')
input_birth = input('birth를 입력하시오')

# 회원 정보를 저장하는 query 생성
signup_query = """
    insert into
    `user_info`
    values
    ( %s, %s, %s, %s )  
"""

# signup_query와 데이터를 이용하여 execute 사용
cursor.execute(signup_query, [input_id, input_pass, input_name, input_birth])

1

- CUD(insert, update, delete)
    - query 작성
        - 데이터가 존재한다면 데이터들을 리스트나 튜플의 형태로 생성
    - execute() 함수를 이용하여 cursor에 질의를 보낸다.
        - cursor에 존재하는 table의 data가 변화
    - DB server와 cursor를 동기화 (.commit())
    - DB server와의 연결을 종료

- R(select)
    - query 작성
        - 데이터가 존재한다면 데이터들을 리스트나 튜플의 형태로 생성
    - execute() 함수를 이용하여 cursor에 질의를 보낸다.
        - cursor에 존재하는 table의 data에 변화는 없음
    - execute() 함수의 결과를 불러와서 변수에 저장(fetchall())
    - DB server와의 연결을 종료

In [70]:
# query문이 select문인가?
test_query = """
    select
    *
    from
"""

In [ ]:
# query문이 select인가?
# 방법1
text_data = 'select'
test_query.strip().lower()[ : len(text_data)] == text_data

True

In [ ]:
#방법2 
test_query.lower().split()[0] == text_data


True

In [ ]:
#방법3
test_query.lower().strip().startswith(text_data)

True

In [ ]:
# CRUD query를 모두 사용할 수 있는 함수 생성
# 매개변수 2개 -> query문, query문에 대입되는 데이터들(가변)
def sql_query(
        _query, *_data_list
):
    # *(가변)은 0개도 포함. 빈리스트 생성 가능하기 때문, 입력할 때는 데이터 묶어줄 필요 없음
    # query문을 실행을 시도한다.
    try:
    # 질의를 보낸다
        cursor.execute(_query, _data_list)
    #execute 함수가 에러가 발생한다면
    except Exception as e:
        print(e)
        # 함수를 강제로 종료하면서 메시지 되돌려준다.
        return "Query Error"
    # _query 는 sql query문이 문자로 입력이 되는 부분
    # _query가 select문인가?
    if _query.lower().strip().startswith('select'):
        result = cursor.fetchall()

    else:
        result = 'Query OK'

    return result

In [71]:
test_query = """
select
*
from
`user_info`
"""

sql_query(test_query)

[{'id': '', 'password': '', 'name': '', 'birth': ''},
 {'id': 'test', 'password': '1234', 'name': 'kim', 'birth': '900101'},
 {'id': 'test0000', 'password': '1234', 'name': 'park', 'birth': '000101'},
 {'id': 'test1234', 'password': '1234', 'name': 'kim', 'birth': '900101'},
 {'id': 'test2', 'password': '1234', 'name': 'park', 'birth': '000101'},
 {'id': 'test9999', 'password': '7890', 'name': 'seo', 'birth': '030101'}]

In [72]:
# 로그인 과정
# 유저가 입력한 id, password를 이용해서 데이터베이스에 데이터가 존재하는가?
input_id = input("id 입력하시오")
input_pass = input("password 입력하시오")

login_query = """
    select
    *
    from
    `user_info`
    where
    `id` = %s
    and
    `password` = %s
"""

login_result = sql_query(login_query, input_id, input_pass)

In [ ]:
login_result

()

In [ ]:
if login_result:
    # login_result에 데이터가 존재한다면 -> 길이가 1인
    print('로그인 성공')
else:
    # login_result에 데이터가 존재하지 않는다면 -> 길이가 0인
    print('로그인 실패')

로그인 실패


In [ ]:
input_id = 'test0000'
input_pass = '1234'
input_name = 'park'
input_birth = '000101'

insert_query = """
    insert into
    `user_info`
    values
    (%s, %s, %s, %s)
"""
sql_query(insert_query, input_id, input_pass, input_name, input_birth)

'Query OK'

In [84]:
# 데이터베이스 서버와 cursor를 동기화하고 서버와의 연결을 종료하는 함수 생성
def commit_db():
    # cursor와 데이터베이스서버와의 동기화
    _db.commit()
    print('Commit 완료')
    # 데이터베이스 서버와의 연결을 종료
    _db.close()
    print('서버와의 연결 종료')



In [85]:
commit_db()

Commit 완료
서버와의 연결 종료


In [154]:
class MyDB():
    # 생성자 함수
    # class 내부에서 사용하려는 변수의 데이터를 대입하는 함수
    # class 생성 될 때 한번만 실행이 되는 함수
    # 입력 받을 데이터는 데이터베이스 서버의 정보 -> 기본값 설정(로컬피씨의 데이터베이스 정보)

    def __init__(
            self, 
            _host = os.getenv('host'),
            _port = int(os.getenv('port')),
            _user = os.getenv('user'),
            _pw = os.getenv('pw'),
            _db_name = os.getenv('db_name')
    ):
        # self.변수를 생성
        # class에서 사용할 서버의 정보를 변수에 저장
        self.host = _host
        self.port = _port
        self.user = _user
        self.pw = _pw
        self.db_name = _db_name

    def sql_query(
            self,
            #위에서 작성한 self 변수들을 써야하기 때문에
            _query,
            *_data_list
    ):
        # _query 매개변수는 기본값이 존재하지 않으므로 필수 입력 공간
        # _data_list는 인자의 개수를 가변으로 받는다. 개수가 0개면 ()을 생성

        # database 서버와의 연결(_db 변수를 생성하여 연결)
        # pymysql.connect 함수는 입력값이 서버의 정보

        try: 
            # self._db라는 변수를 확인
            self._db
            # 해당 변수가 선언이 되어있지 않다면 NameError 발생
        except:
            # self._db
            self._db = pymysql.connect(
                host = self.host,
                port = self.port,
                user = self.user,
                password = self.pw,
                db = self.db_name
            )

        # self._db 이용하여 cursor 생성
        # pymysql의 cursor 들 중에 딕셔너리 커서를 사용하여 만들기
        self.cursor = self._db.cursor(pymysql.cursors.DictCursor)

        #_query와 _data_list를 이용하여 self.cursor에 질의 보낸다.
        # 질의를 보내는 과정에서 문제가 발생하면 예외처리를 한다.
        try:
            self.cursor.execute(_query, _data_list)
        except Exception as e:
            # 무슨 에러인지 출력
            print(e)
            # query 문에서 문제가 발생했으니 
            return 'Query Error'
        
        if _query.lower().strip().startswith('select'):
            # query 문이 select문이라면
            result = self.cursor.fetchall()
        else:
            # select문이 아닌경우
            result = "Query OK"
        return result


    def commit_db(self):
        try:
            self._db.commit()
            print('Commit 완료')
            self._db.close()
            print('서버와의 연결 종료')
            # 변수를 삭제
            del self._db
        except Exception as e:
            print(e)
            # self._db와 self._cursor가 생성되지 않은 상황
            print('''데이터베이스 서버와의 연결이 없습니다. sql_query() 함수를 이용하여 서버와의 연결을 해주세요''')



##
# 함수에서 전역변수에 접근하려면 globals를 사용해야하는 것처럼
# class에서 . class 안에서의 공간에서 사용하려는 변수에는 self를 붙인다.
# 일반적으로 함수 내부의 변수는 함수에서만 사용 가능한데,
# self를 붙이면 class내부의 다른 함수에서도 사용이 가능하다.

In [155]:
# MyDB class 생성하는데 생성자 함수의 매개변수는 기본값으로 사용
# 로컬피씨에 있는 데이터베이스에 연결
db1 = MyDB()

In [113]:
# db2 = MyDB(
#     '192.168.50.40',
#     3306,
#     'test2',
#     '1234',
#     'multicam'
# )

In [157]:
select_query = """
    select
    *
    from
    `user_info`
"""

In [158]:
db1.sql_query(select_query)

[{'id': '', 'password': '', 'name': '', 'birth': ''},
 {'id': 'test', 'password': '1234', 'name': 'kim', 'birth': '900101'},
 {'id': 'test0000', 'password': '0123', 'name': 'park', 'birth': '000101'},
 {'id': 'test1234', 'password': '0123', 'name': 'kim', 'birth': '900101'},
 {'id': 'test2', 'password': '1234', 'name': 'park', 'birth': '000101'},
 {'id': 'test9999', 'password': '0123', 'name': 'seo', 'birth': '030101'}]

In [116]:
# db2.sql_query(select_query)

In [159]:
update_query = """
    update
    `user_info`
    set
    `password` = "0123"
    where
    `id` = 'test2'
"""

In [168]:
db1.sql_query(update_query)

(0, '')


'Query Error'

In [147]:
db1.commit_db()

Commit 완료
서버와의 연결 종료


In [148]:
db1.sql_query(select_query)

[{'id': '', 'password': '', 'name': '', 'birth': ''},
 {'id': 'test', 'password': '1234', 'name': 'kim', 'birth': '900101'},
 {'id': 'test0000', 'password': '0123', 'name': 'park', 'birth': '000101'},
 {'id': 'test1234', 'password': '0123', 'name': 'kim', 'birth': '900101'},
 {'id': 'test2', 'password': '1234', 'name': 'park', 'birth': '000101'},
 {'id': 'test9999', 'password': '0123', 'name': 'seo', 'birth': '030101'}]

In [161]:
import database

In [162]:
# database 모듈 안에 있는 MyDB 생성
mod_db = database.MyDB()

In [164]:
# mod_db에 입력이 되는 데이터베이스 서버의 정보는? -> 로컬
select_query2 = """
    select * from `user_info`
"""
db_data = mod_db.sql_query(select_query2)

In [165]:
pd.DataFrame(db_data)

,id,password,name,birth
0,,,,
1,test,1234,kim,900101
2,test0000,0123,park,000101
3,test1234,0123,kim,900101
4,test2,1234,park,000101
5,test9999,0123,seo,030101


In [166]:
# database.py 파일을 python 환경변수에 이동

# 환경 변수의 목록을 확인하기 위해 라이브러리 하나 호출
import sys

In [167]:
sys.path

['/Users/eunseo/Documents/data_boot/lecture/code/week3',
 '/opt/homebrew/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python313.zip',
 '/opt/homebrew/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python3.13',
 '/opt/homebrew/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python3.13/lib-dynload',
 '',
 '/Users/eunseo/Documents/data_boot/venv/lib/python3.13/site-packages']